# Video segments: frames, shots and scenes

_Video time segmentation_ (비디오 시간 분할)은 분석과 자동화를 위한 비디오 콘텐츠의 잠재력을 최대한 활용하는 데 도움이 되는 중요한 데이터 준비 단계입니다. 비디오를 의미 있는 세그먼트로 나눔으로써 콘텐츠의 구조와 맥락을 더 잘 이해할 수 있으며, 다음과 같은 다양한 응용이 가능합니다:

* 비디오 내의 주요 events, scenes, chapters 식별
* 광고 마커나 chapter markers와 같은 메타데이터 삽입
* 새로운 목적을 위해 관련 클립이나 세그먼트 재사용
* 비디오의 특정 부분에 고급 분석 및 Foundation 모델 적용

![Video file decomposed into frames, shots and scenes](./static/images/01-visual-segments.jpg)

***Figure:** frames, shots (번호가 매겨짐), scenes (색상으로 구분됨)으로 분해된 비디오 파일*


이 노트북에서는 시각적 단서를 사용하여 비디오를 더 작은 세그먼트로 분해하는 기술을 탐구할 것입니다. 구체적으로 다음과 같은 작업을 수행합니다:

* 시각적 분석을 사용하여 비디오를 frames, shots, scenes로 분해
* Amazon Bedrock의 Foundation Models (FMs)에 프롬프트로 사용할 수 있는 비디오 세그먼트의 composite images 생성
* 비디오 콘텐츠를 이해하기 위한 Foundation Model과의 prompt engineering 연습

이 노트북을 마치면 비디오의 세그먼테이션 컬렉션을 갖게 될 것이며, 이는 비디오 자산의 추가 분석, 자동화 및 재사용을 위한 기초가 될 수 있습니다.

이 노트북의 출력은 이후 워크숍의 사용 사례 섹션에서 사용될 것입니다.

<div class="alert alert-block alert-info">
💡 워크숍의 이 부분에서는 세그먼테이션 작업을 수행하기 위해 몇 가지 Python 라이브러리를 사용할 것입니다. 코드를 자세히 살펴보고 싶다면 이 프로젝트의 <b>lib/</b> 폴더에 라이브러리가 있습니다. prompt engineering과 사용 사례 해결에 더 많은 시간을 할애하기 위해 이 섹션에서는 모든 구현 세부사항을 다루지는 않을 것입니다.
</div>

<div class="alert alert-block alert-info">
💡 비디오 세그먼테이션은 temporal dimension 또는 spatial dimension을 따라 수행될 수 있습니다. 이 노트북의 맥락에서 "segmentation"이라는 용어는 항상 <i>temporal (time) segmentation</i>을 의미합니다.
</div>

### 주요 용어 및 정의

노트북에서 사용되는 용어의 정의를 확인하고 싶을 때 이 섹션을 참조할 수 있습니다.

- **Frame** - 비디오 콘텐츠에서 추출한 프레임 이미지
- **Frame sampling** - 비디오에서 대표적인 프레임들의 부분집합을 선택하는 것
- **Shot** - 하나의 액션을 정의하는 두 편집 또는 컷 사이의 연속적인 프레임 시퀀스
- **Scene** - 특정 위치와 시간에서 발생하는 연속적인 액션 시퀀스로, 일련의 shots으로 구성됨
- **Frame accurate timestamp** - 특정 프레임에 매핑될 수 있는 타임스탬프. Frame accurate timestamps는 비디오 요소의 동기화에 유용함

### 워크플로우

이 실습의 목적은 frame, shot, scene 수준에서 비디오의 시각적 요소를 다루는 실습과 비디오 클립을 나타내는 프레임 시퀀스로 prompt engineering을 연습하는 것입니다. 활동 전반에 걸쳐 SageMaker 노트북에서 AWS 서비스를 사용하여 작업할 것입니다.

![scene detection and contextualization workflow](./static/images/01-scenes-shots-workflow-w-ouputs-drawio.png)

<div class="alert alert-block alert-info">
💡 이 Jupyter 노트북의 왼쪽 탐색 패널에서 목록 아이콘을 클릭하면 노트북의 개요와 현재 위치를 볼 수 있습니다.
</div>

## Prerequisites

### Import python packages

In [ ]:
from pathlib import Path
import os
import json
import boto3
from termcolor import colored
from IPython.display import JSON
from IPython.display import Video
from IPython.display import Image as DisplayImage
from lib.frames import VideoFrames
from lib.shots import Shots
from lib.scenes import Scenes
from lib.transcript import Transcript
from lib import util
import requests
from tqdm import tqdm
from lib import frame_utils
from PIL import Image, ImageDraw, ImageFont
import time
from functools import cmp_to_key

### 이전 노트북에서 저장된 값 검색하기

이 노트북을 실행하려면 패키지 종속성을 설치하고 SageMaker 환경에서 일부 정보를 수집한 이전 노트북 [00_prerequisites.ipynb](./00-prequisites.ipynb)을 실행했어야 합니다.

In [ ]:
# Get variables from the previous notebook
%store -r

In [ ]:
%store

### Download the sample video

In [ ]:
def download_video(url: str, output_path: str) -> str:
    """
    Download test video if not already present
    """
    
    if os.path.exists(output_path):
        print(f"Video already exists at {output_path}")
        return output_path
    
    print("Downloading test video...")
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(output_path, 'wb') as file, tqdm(
        desc=output_path,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as pbar:
        for data in response.iter_content(chunk_size=1024):
            size = file.write(data)
            pbar.update(size)
            
    return

### 샘플 비디오 다운로드

* Meridian, 2016, Mystery from [Netflix](https://opencontent.netflix.com/) - 이 콘텐츠는 [Creative Commons Attribution 4.0 International Public License](https://creativecommons.org/licenses/by/4.0/legalcode) 하에서 사용 가능합니다.

다음 셀은 실습의 나머지 부분에서 사용될 비디오를 선택합니다. 기본값은 Meridian입니다. 변경하고 싶다면 `video['path']`를 설정하는 줄을 수정하여 다른 비디오를 사용할 수 있습니다.

<div class="alert alert-block alert-info">
일부 시각화 선택이 해당 콘텐츠에 잘 작동하도록 되어 있으므로, 첫 번째 워크숍에서는 <b>Meridian</b>으로 진행하는 것을 권장합니다.
</div>

In [ ]:
video = {}

# Video Alternatives
# Drama
MERIDIAN='Netflix_Open_Content_Meridian.mp4'

video["path"] = MERIDIAN
video["url"] = f"https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/7db2455e-0fa6-4f6d-9973-84daccd6421f/{video['path']}"

video["output_dir"] = Path(video["path"]).stem

download_video(video["url"], video["path"])

Video(url=video["url"], width=640, height=360, html_attributes="controls muted autoplay")

# 프레임 샘플링

이 섹션에서는 392×220 픽셀 해상도로 초당 한 프레임씩 비디오 프레임을 추출할 것입니다. 이러한 설정은 수많은 실험을 통해 시각적 품질과 계산 효율성을 최적화한 것입니다. 이 과정에서 초당 한 프레임을 샘플링합니다.

<div class="alert alert-block alert-info">
💡 392×220 픽셀 해상도는 우리가 선택한 Foundation Model인 Anthropic Claude Sonnet 3에 제시할 수 있는 프레임 수를 최적화하면서도 사용 사례에 필요한 세부 정보 수준을 유지하도록 선택되었습니다. 다른 사용 사례의 경우 더 낮은 비용이나 더 높은 품질을 위해 더 높거나 낮은 해상도를 사용할 수 있습니다.
</div>

<div class="alert alert-block alert-info">
💡 초당 한 프레임 샘플링은 여기서 사용된 콘텐츠에 적합한 설계 선택이지만, 스포츠와 같은 고동작, 고프레임 레이트 비디오나 뉴스룸 영상과 같은 더 정적인 비디오에 맞게 조정될 수 있습니다.
</div>

### 비디오에서 프레임 추출하기

프레임 수준에서 비디오 작업을 하기 위해 Python 패키지인 [VideoFrames](./lib/frames.py)를 사용할 것입니다. 이 패키지는 GitHub에서 사용할 수 있습니다. VideoFrames의 main 메서드를 호출하면 머신 러닝을 통한 프레임 기반 분석을 위해 비디오를 준비하는 다음 단계들을 수행합니다:

1. 지정된 프레임 레이트로 샘플링하여 비디오에서 프레임을 추출하고, 결과 이미지를 `./<video name>/frames/` 폴더에 저장합니다.
2. 결과 프레임 메타데이터는 각 프레임에 대해 다음 속성들을 포함합니다:

* **timestamp_millis** — 비디오에서 프레임이 나타나는 시간을 밀리초 단위로 표시한 타임스탬프. 이 타임스탬프를 사용하여 비디오 분석 결과를 비디오 타임라인과 연관시킬 것입니다.
* **image_file** — `frames` 폴더에 있는 이미지의 위치
* **id** - 고유한 프레임 id

한번 시도해 보겠습니다.

⏳ 프레임 생성에는 몇 분 정도의 처리 시간이 소요됩니다.

In [ ]:
# Number of frames to sample per second of video
FRAME_SAMPLING_RATE = 1

video["frames"] = VideoFrames(video["path"], session['bucket'], max_res=(392, 220), sample_rate_fps=FRAME_SAMPLING_RATE, force=False)

In [ ]:
display(JSON(video["frames"].frames[0], root="first frame"))


### 추출된 프레임 시각화하기

다음으로, 비디오에서 추출된 프레임들을 시각화해 보겠습니다.

<div class="alert alert-block alert-info">
🤔 프레임에서 시각적 패턴을 발견할 수 있나요? 프레임을 기반으로 비디오에 몇 개의 shots이 있는지 예측할 수 있나요?
</div>

In [ ]:
video['frames'].display_frames(start=0, end=len(video['frames'].frames))

# Shots 감지

shot은 하나의 액션을 정의하는 두 편집 또는 컷 사이의 연속적인 프레임 시퀀스입니다. 일반적으로 shot은 단일 카메라 위치를 나타내지만, 때로는 패닝이나 줌과 같은 카메라 움직임을 포함할 수 있습니다. 동일한 shot에 속하는 프레임들은 유사해야 합니다. 따라서, shot 감지를 구현하는 한 가지 방법은 Amazon [Titan Multimodal Embeddings](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-multiemb-models.html)와 같은 이미지 임베딩을 사용하여 유사한 프레임들을 shots으로 그룹화하는 것입니다. 하지만 샘플링된 프레임으로 작업하고 있기 때문에, shots의 타임스탬프 정확도는 프레임 샘플링 속도에 의해 제한될 것입니다.

광고 삽입, 편집, 검색과 같은 사용 사례에서는 shots이 시작하고 끝나는 정확한 프레임을 식별하는 **frame accurate** 타임스탬프를 사용하는 것이 이상적입니다. [Amazon Rekognition's Segment API](https://docs.aws.amazon.com/rekognition/latest/dg/segments.html)는 비디오 콘텐츠에서 기술적 단서와 shot 경계를 자동으로 감지하여 각 shot 경계에 대한 frame accurate 타임스탬프를 제공하는 비디오 분석 서비스입니다.

[Shots](lib/shots.py) Python 라이브러리를 사용하여 Amazon Rekognition's Shot Segment API로 shots을 생성하고, 이전 단계에서 샘플링한 프레임에 shots을 매핑합니다. 이 결과를 사용하여 shots의 큰 그룹을 scenes로 구성할 것입니다.

## Shots 라이브러리

<div class="alert alert-block alert-info">
<b>참고</b>: shot 감지를 위한 코드는 더 자세히 살펴보고 싶다면 이 프로젝트의 <b>lib</b> 폴더에 있지만, 이 연습의 목적은 shot의 개념을 이해하는 것입니다.
</div>

[lib/shots.py](lib/shots.py)를 열려면 링크를 클릭하세요

## Shot 감지 실행하기

In [ ]:
video["shots"] = Shots(video["frames"], method="RekognitionShots")

print(f"Number of shots: {len(video['shots'].shots)} from {len(video['frames'].frames)} frames")

결과에서 샘플 shot의 메타데이터를 잠시 살펴봅니다. 각 shot은 다음을 포함합니다:

* **method** - 프레임을 shots으로 그룹화하는 데 사용된 방법. 가능한 값은 `SimilarFrames` 또는 `RekognitionShots`입니다
* **start_ms** - shot의 시작 타임스탬프
* **end_ms** - shot의 종료 타임스탬프
* **duration_ms** - shot의 지속 시간
* **video_asset_dir** - 이 비디오에 대해 수집된 메타데이터의 위치
* **start_frame_id** - shot이 시작되는 프레임
* **end_frame_id** - shot이 끝나는 프레임
* **composite_images** - shots의 프레임을 포함하는 동일한 크기의 프레임 그리드 시리즈. Composite images는 shot에 대한 인사이트를 생성하기 위해 멀티모달 foundation models의 입력으로 사용될 수 있습니다.


Shot 1의 메타데이터 표시

In [ ]:
display(JSON(video["shots"].shots[1]))

`Shots` 메서드는 각 shot의 프레임들로 구성된 composite images 세트를 생성합니다. 이러한 composite images는 나중에 Amazon Bedrock의 Anthropic Claude Sonnet 3에 입력으로 사용되어 shots에서 무슨 일이 일어나고 있는지 이해하기 위한 추론을 생성하는 데 사용될 것입니다. 지금은 shots를 시각화하기 위해 결과로 나온 composite images를 살펴볼 수 있습니다.

Shot 1의 composite images 표시

In [ ]:
shot = video["shots"].shots[1]
for idx, composite_image in enumerate(shot['composite_images']):
    
    print (f'\nShot {shot["id"] } Composite image file { idx+1 } of { len(shot["composite_images"]) }: { composite_image["file"] }\n')
    display(DisplayImage(filename=composite_image['file']))


scene 감지로 넘어가기 전에, 생성된 모든 shots를 살펴보겠습니다.

<div class="alert alert-block alert-info">
💡 출력 상자의 스크롤 바를 사용하여 shots를 볼 수 있습니다.
</div>

In [ ]:
# visualize the shots
for counter, shot in enumerate(video["shots"].shots):
    print(f'\nSHOT {counter}: frames {shot["start_frame_id"] } to {shot["end_frame_id"] } =======\n')
    video['frames'].display_frames(start=shot["start_frame_id"], end=shot["end_frame_id"]+1)
    
    # ALTERNATIVE: Display composite images for each shot
    #for image_file in shot['composite_images']:
    #    display(DisplayImage(filename=image_file['file'], height=75))
    

<div class="alert alert-block alert-info">
💡 위 셀에서 shots를 보려면 출력 상자의 스크롤 바를 사용하세요. 출력 셀의 오른쪽 하단 모서리를 드래그하여 크기를 늘릴 수도 있습니다.
</div>


🤔 shots 목록을 살펴보면서, 예상치 못한 결과가 있는 세그먼트를 발견하셨나요? 만약 발견하셨다면, 자동 비디오 세그먼테이션에서 까다로운 상황들을 발견하신 것일 수 있습니다. 이러한 상황들은 다음과 같습니다:
* 롤링 크레딧, 자동차 등을 포함한 피사체의 움직임
* 패닝 shots와 줌 shots 형태의 카메라 움직임
* 페이드 및 기타 전환 효과

scene 감지로 넘어가기 전에, 인접한 shots 몇 개를 재생하여 비디오 클립으로 어떻게 보이는지 관찰해 보겠습니다. `shot[12]`를 재생하세요.




In [ ]:
start = video['shots'].shots[12]['start_ms']/1000
end = video['shots'].shots[12]['end_ms']/1000
shot_url = f'{video["url"]}#t={start},{end}'
Video(url=shot_url, width=640, height=360)

Play `shot[13]`

In [ ]:
start = video['shots'].shots[13]['start_ms']/1000
end = video['shots'].shots[13]['end_ms']/1000
shot_url = f'{video["url"]}#t={start},{end}'
Video(url=shot_url, width=640, height=360)

🤔 Meridian으로 워크숍을 진행하고 있다면, 이 shots는 같은 방에서 shots에 걸쳐 대화를 나누는 두 사람을 보여줍니다. 이 두 shots는 같은 설정에 있기 때문에, 더 높은 수준의 그룹으로 함께 묶여야 합니다. 다음 섹션에서는 scene의 전체적인 모습을 파악하기 위해 시각적 정보를 기반으로 shots를 함께 그룹화할 것입니다.

# Scenes 감지

개별 카메라 shots를 식별한 후에도, 같은 설정을 묘사하는 의미론적으로 유사한 shots가 있을 수 있습니다. 이들을 distinct scenes로 더 클러스터링하기 위해, 인접 프레임을 넘어서 프레임 비교를 확장합니다. 확장된 시간 창에서 유사한 프레임들을 살펴봄으로써, 같은 연속적인 scene의 일부일 가능성이 있는 shots를 식별할 수 있습니다. 주어진 시간 창 내의 모든 프레임 간 쌍별 유사도 점수를 계산합니다. 특정 임계값 이상의 유사도 점수를 가진 프레임들은 같은 scene 그룹의 일부로 간주됩니다. 이 프로세스는 shot의 모든 프레임에 대해 재귀적으로 수행됩니다.

<div class="alert alert-block alert-info">
💡 시간 창 크기와 유사도 임계값은 scene 경계 감지의 정확도에 상당한 영향을 미칠 수 있는 매개변수입니다. 우리의 예시에서는 30초의 시간 창과 0.80 유사도 임계값이 비디오 샘플에서 가장 좋은 scene 클러스터링 결과를 제공했지만, 이는 조정될 수 있습니다.
</div>

우리는 다음 그림과 같이 TME를 다시 사용하여 모든 비디오 프레임을 인덱싱하고 shot 정보와 타임스탬프와 함께 임베딩을 벡터 데이터베이스에 저장함으로써 scene 그룹화를 수행합니다.

![shots-to-scenes.png](./static/images/01-vectorization.jpg)

그런 다음 이 인덱싱된 프레임 코퍼스에 대해 재귀적 유사도 검색을 수행합니다. 각 프레임에 대해, 양방향으로 3분의 시간 창 내에서 벡터 표현을 기반으로 80% 이상의 맥락적 유사성을 가진 다른 모든 프레임을 찾습니다. 이러한 매우 유사한 프레임들의 shot 정보가 기록됩니다. 이 프로세스는 맥락적으로 유사한 shots를 컴파일하기 위해 shot의 모든 프레임에 대해 반복됩니다. 이 프로세스는 모든 shots에 걸쳐 반복되며, 컴파일된 결과는 다음 예시와 같습니다:

    shot 1 –> 2, 3, 4
    shot 2 –> 1, 3
    shot 3 –> 2, 4, 5
    shot 7 –> 8, 9

마지막으로, 다음과 같이 상호 매우 유사하다고 식별된 shots를 distinct scene 그룹으로 그룹화하는 축소 프로세스를 실행합니다:

    shot 1, 2, 3, 4, 5 → scene 1
    shot 7, 8, 9 → scene 2

이를 통해 초기에 감지된 shot 경계를 시각적 및 시간적 일관성을 기반으로 더 높은 수준의 의미론적 scene 경계로 세그먼트화할 수 있습니다. 전체 프로세스는 다음 다이어그램에 설명되어 있습니다.

### 프레임 임베딩 생성

이미지 임베딩은 이미지의 필수적인 특징과 특성을 캡처하는 수치적 표현(벡터)입니다. 이러한 임베딩을 통해 이미지에 대한 수학적 연산을 수행하고 인간의 시각적 인식과 일치하는 방식으로 비교할 수 있습니다.

Amazon Bedrock의 [Amazon Titan Multimodal Embeddings](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-multiemb-models.html)를 사용하여 비디오의 각 프레임에 대한 이미지 임베딩을 생성할 것입니다. 이 작업을 수행하기 위해 [lib/frames.py](./lib/frames.py)의 헬퍼 함수를 사용할 것입니다.

계산된 프레임 임베딩은 `video` 변수에 저장된 `Frames` 객체의 각 프레임에 추가될 것입니다.

`Frames` 클래스의 `make_titan_multimodal_embeddings()` 메서드를 호출하면 프레임 임베딩을 생성하고 각 프레임의 메타데이터와 함께 저장합니다.

<div class="alert alert-block alert-info">
이 시점에서 <b>AccessDenied</b> 오류가 발생하면, Amazon Bedrock 콘솔에서 Amazon Titan Multimodal Embeddings와 Anthropic Claude Sonnet 3에 대한 모델 액세스를 활성화하는 단계를 완료했는지 확인하세요.
</div>

<div class="alert alert-block alert-info">
⏳ 샘플 비디오에 대한 임베딩 생성은 2-5분 정도 소요됩니다. 속도를 높이기 위해 미리 계산된 임베딩을 로드할 것입니다.
</div>

In [ ]:
# workshop FASTPATH setting uses pre-calculated embeddings for the video, set FASTPATH=false to regenerate embeddings
FASTPATH = True
if FASTPATH:
    video['frames'].load_titan_multimodal_embeddings()
else:
    video['frames'].make_titan_multimodal_embeddings()

다음 셀을 사용하여 첫 번째 프레임의 메타데이터를 출력하고 `titan_multimodal_embedding` 속성을 검사합니다. 이것은 Titan Multimodal Embeddings 모델의 `amazon.titan-embed-image-v1` 버전에 대한 벡터 공간에서 프레임의 내용을 인코딩하는 큰 벡터입니다. 동일한 모델 버전을 사용하여 인코딩된 다른 벡터들과 이 벡터를 비교할 때, 우리는 그들이 유사한지 판단할 수 있습니다.

In [ ]:
display(JSON(video["frames"].frames[0], root="first frame"))

다음으로, 임베딩을 사용하여 처음 몇 개의 프레임을 비교해 보겠습니다. 먼저, 샘플링된 프레임 몇 개를 출력합니다.

In [ ]:
video['frames'].display_frames(start=0, end=10)

프레임을 비교하기 위해서는 임베딩을 비교하는 방법이 필요합니다. Python numpy 패키지를 사용하여 [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) 함수를 구현할 것입니다.

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
    cos_sim = dot(a, b) / (norm(a) * norm(b))
    return cos_sim

## 다음으로, 몇 개의 프레임을 비교해 보겠습니다.

첫 번째 검은색 프레임을 콘텐츠가 있는 프레임과 비교해 보겠습니다. Meridian의 경우, 도시 거리 전경이 있는 두 번째 프레임을 선택할 수 있습니다. 예상대로 이 프레임들은 매우 유사하지 않기 때문에 유사도 점수가 낮습니다.

In [ ]:
frms = video['frames'].frames
cosine_similarity(frms[0]['titan_multimodal_embedding'], frms[2]['titan_multimodal_embedding'])

이제 비슷해 보이는 두 프레임을 비교해 보겠습니다. Meridian의 경우, 두 번째 프레임과 세 번째 프레임을 비교할 수 있습니다. 이 프레임들의 주요 차이점은 "Los Angeles 1947"이라는 글자뿐이므로 유사도 점수가 더 높아야 합니다.

In [ ]:
cosine_similarity(frms[1]['titan_multimodal_embedding'], frms[2]['titan_multimodal_embedding'])

### FAISS 벡터 저장소 채우기

특정 프레임과 유사한 모든 프레임을 한 번에 찾기 위해 단일 검색 명령을 사용할 수 있도록 로컬 FAISS 벡터 저장소를 사용할 것입니다. 우리의 검색 함수는 이전 섹션에서 살펴본 것과 동일한 코사인 유사도 방법을 사용할 것입니다. AWS에는 벡터 저장소로 사용할 수 있는 여러 데이터베이스가 있습니다. 인기 있는 선택 중 하나는 [Amazon Opensearch](https://aws.amazon.com/opensearch-service/)입니다.

In [ ]:
video['frames'].make_vector_store()

### 벡터 저장소를 사용한 유사도 검색 테스트

벡터 저장소를 사용하여 비디오의 두 번째 프레임과 유사한 프레임들을 찾아보겠습니다. 이것은 "Los Angeles 1947"라는 단어를 표시하는 시퀀스의 첫 번째 프레임입니다. 프레임 검사를 기반으로, 3개의 _인접한_ 유사 프레임을 얻어야 하지만, 인접하지 않은 여러 유사 프레임들도 있습니다.

우리의 유사도 검색 함수는 벡터 공간에서 [K nearest neighbors](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)를 결정하기 위해 [코사인 유사도 함수](https://en.wikipedia.org/wiki/Cosine_similarity)를 사용합니다. 유사도 검색 결과를 조정하기 위해 조정할 수 있는 두 가지 매개변수가 있습니다:

* MIN_SIMILARITY는 유사도 임계값입니다.
* TIME_RANGE는 입력 프레임으로부터 프레임을 비교할 최대 시간 범위입니다.

결과가 어떻게 변하는지 느껴보기 위해 이러한 매개변수의 다른 값들을 시도해볼 수 있습니다. 결과를 시각화하는 데 도움이 되도록 프레임 0-19가 이후에 표시됩니다. 다음은 시도해볼 만한 좋은 값들입니다:

* MIN_SIMILARITY = .85, TIME_RANGE = 30
* MIN_SIMILARITY = .70, TIME_RANGE = 30
* MIN_SIMILARITY = .80, TIME_RANGE = 3

In [ ]:
MIN_SIMILARITY = .80
TIME_RANGE = 30
FRAME_ID = 1
video['frames'].search_similarity(FRAME_ID, min_similarity = MIN_SIMILARITY, time_range = TIME_RANGE)

In [ ]:
video['frames'].display_frames(start=0, end=20)

### Scenes 라이브러리

<div class="alert alert-block alert-info">
<b>참고</b>: scene 감지를 위한 코드는 더 자세히 살펴보고 싶다면 이 프로젝트의 <b>lib</b> 폴더에 있지만, 이 연습의 목적은 scene의 개념을 이해하는 것입니다.
</div>

[lib/scenes.py](lib/scenes.py)를 열려면 링크를 클릭합니다.

### Scene 감지 실행

이제 유사한 shots를 찾기 위해 모든 shots의 프레임에 대해 이 유사도 검색을 적용해 보겠습니다. TIME_RANGE 내에서 shots가 유사하다면, 같은 scene으로 그룹화될 것입니다.

In [ ]:
MIN_SIMILARITY = .80

TIME_RANGE = 30

video['scenes'] = Scenes(video['frames'], video['shots'].shots, MIN_SIMILARITY, TIME_RANGE)


### Scenes 시각화

이제 생성된 composite images를 사용하여 일부 scenes를 시각화해 보겠습니다. 일부 scenes에는 하나 이상의 composite image가 있을 수 있습니다.

🤔 scenes가 shots를 의미 있는 방식으로 그룹화했다고 생각하시나요? 변경하고 싶은 부분이 있나요?

In [ ]:
# visualize the scenes
for counter, scene in enumerate(video["scenes"].scenes):
    print(f'\nScene {counter}: frames {scene["start_frame_id"] } to {scene["end_frame_id"] } =======\n')
    video['frames'].display_frames(start=scene["start_frame_id"], end=scene["end_frame_id"]+1)

<div class="alert alert-block alert-info">
💡 위의 출력 상자에서 scenes를 보려면 스크롤 바를 사용합니다. 출력 셀의 오른쪽 하단 모서리를 드래그하여 크기를 늘릴 수도 있습니다.
</div>

마지막으로, 인접한 scenes 몇 개를 재생하여 비디오 클립으로 어떻게 보이는지 관찰해 보겠습니다. 비디오 세그먼트를 재생할 때 한 scene에서 다음 scene으로 넘어갈 때 _비디오와 오디오_의 전환에 주의를 기울이세요. scenes는 시각적 정보만을 기반으로 생성되기 때문에, scene 경계에서 클립을 만들면 오디오가 잘릴 수 있습니다. 다음 노트북에서는 비디오의 오디오 세그먼테이션을 살펴볼 것입니다.

In [ ]:
start_scene = 10
start = video['scenes'].scenes[start_scene]['start_ms']/1000
end = video['scenes'].scenes[start_scene]['end_ms']/1000
print(f"scene { start_scene } duration: {video['scenes'].scenes[start_scene]['duration_ms']/1000} seconds\n")
print(f"start time: { start } end time: {end} seconds\n")
shot_url = f'{video["url"]}#t={start},{end}'
Video(url=shot_url, width=640, height=360)

다음 scene을 재생합니다.

In [ ]:
start_scene = start_scene + 1
start = video['scenes'].scenes[start_scene]['start_ms']/1000
end = video['scenes'].scenes[start_scene]['end_ms']/1000
print(f"scene { start_scene } duration: {video['scenes'].scenes[start_scene]['duration_ms']/1000} seconds\n")
print(f"start time: { start } end time: {end} seconds\n")
shot_url = f'{video["url"]}#t={start},{end}'
Video(url=shot_url, width=640, height=360)

🤔 우리는 방금 scene 10과 11을 재생했습니다. 비디오의 초점이 해변과 바다로 전환될 때 시각적 신호에 따라 scene이 변경되는 것을 눈치채셨을 수 있습니다. _하지만_, 이 scene 변경은 경찰 디스패처가 무전으로 말하는 오디오 중간에 발생합니다. 클립을 만들기 위해 비디오의 깔끔한 중단점을 식별하고 싶다면, 시각적 콘텐츠뿐만 아니라 오디오도 고려해야 할 것입니다. 이 주제는 워크숍의 다음 부분인 [01B Combining Audio and Video Segments](./01B-combining-audio-and-video.ipynb)에서 살펴볼 것입니다. 하지만 계속 진행하기 전에, 비디오 세그먼트를 사용한 첫 번째 prompt engineering 탐구를 해보겠습니다.

# 비디오 세그먼트로 멀티모달 Foundation Model 프롬프팅하기

![Prompt engineering with frame sequences](./static/images/01-prompt-engineering-with-frame-sequences.jpg)


이 섹션에서는 이전 단계에서 생성한 비디오 세그먼트에 대해 몇 가지 프롬프트를 실행해볼 수 있습니다. 우리는 프롬프트에 이미지와 텍스트를 모두 받을 수 있는 Foundation Model(FM)인 Anthropic Claude Sonnet 3.5를 사용할 것입니다. 다음 코드는 Anthropic Conversations API를 사용하여 프롬프트를 구성하기 위한 몇 가지 헬퍼 함수를 설정합니다.

In [ ]:
import base64
from botocore.exceptions import ClientError

MAX_RETRIES = 50
INITIAL_BACKOFF = 5
bedrock_client = boto3.client(service_name="bedrock-runtime")

def invoke_model_with_retry(body, modelId, accept, contentType):
    retries = 0
    backoff = INITIAL_BACKOFF

    while retries < MAX_RETRIES:
        try:
            response = bedrock_client.invoke_model(
                body=body, modelId=modelId, accept=accept, contentType=contentType
            )
            return response
        except ClientError as e:
            error_code = e.response['Error']['Code']
            print(f"Error: {error_code}. Retrying in {backoff} seconds...")
            time.sleep(backoff)
            retries += 1
            backoff += 1
    
    raise Exception("Max retries reached. Unable to invoke model.")

def generate_segment_description(images, prompt):
    """
    Generate a natural language description of a video shot using LLM in Amazon Bedrock
    Args:
        shot - Dictionary containing shot information including:
                - id: unique identifier for the shot
                - start_ms: start time of the shot in milliseconds
                - end_ms: end time of the shot in milliseconds
                - composite_images: visual representation that combine multiple frames from a single shot into one image
              
    Returns:
        response_body - String containing the generated description of the visual content in the shot based on the analyzed frames
    """


    # model_id = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
    # model_id = "anthropic.claude-3-haiku-20240307-v1:0"
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    accept = "application/json"
    content_type = "application/json"
    
        
    body = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            }
        ],
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
    }


    with open(f"{shot['composite_images'][0]['file']}", "rb") as image_file:
        file_content = image_file.read()
        base64_image_string = base64.b64encode(file_content).decode()
        body["messages"][0]["content"].append({
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base64_image_string,
            },
        })

    open_images = []
    for image in images:
        with open(image['file'], "rb") as image_file:
            file_content = image_file.read()
            base64_image_string = base64.b64encode(file_content).decode()
            body["messages"][0]["content"].append({
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/png",
                    "data": base64_image_string,
                },
            })

    # close the images
    for image in open_images:
        image.close()
    
    response = invoke_model_with_retry(
        body=json.dumps(body), modelId=model_id, accept=accept, contentType=content_type
    )
    response_body = json.loads(response["body"].read())
    response_body = response_body["content"][0]["text"]

    return response_body

우리는 Anthropic Claude의 최대 해상도를 가진 이미지에 프레임들을 패킹할 것입니다. 이 실습에서 생성한 모든 scene과 shot에는 폴더에 저장된 composite image 세트가 있습니다. 이 셀을 실행하여 폴더 이름을 얻습니다. `shots`와 `scenes` 하위 폴더에는 이 실습에서 생성한 composite images가 포함되어 있습니다.

In [ ]:
video["output_dir"]

몇 개의 이미지를 열어서 어떻게 보이는지 확인해봅니다. 다음 셀에서는 composite images를 사용하여 비디오 세그먼트를 이해하기 위한 프롬프트를 몇 개 만들 것입니다.

1. 원하는 shot이나 scene의 `composite_images` 속성으로 `IMAGES` 변수를 설정할 수 있습니다.
2. PROMPT의 지시사항을 변경하여 결과가 어떻게 바뀌는지 볼 수도 있습니다.

이후 실습에서는 다양한 사용 사례를 지원하기 위해 비디오 세그먼트를 맥락화하는 데 유사한 프롬프트를 사용할 것입니다.

다음은 시도해볼 수 있는 대체 IMAGES입니다. 아래 셀의 `IMAGES` 할당을 대체하기 위해 복사하여 붙여넣습니다.:

 ```
    IMAGES = video['scenes'].scenes[15]['composite_images']
    IMAGES = video['scenes'].scenes[20]['composite_images']
    IMAGES = video['shots'].shots[92]['composite_images']
    IMAGES = video['shots'].shots[61]['composite_images']
    IMAGES = video['shots'].shots[35]['composite_images']
```

# 📝 Prompt 예시

> ```
> 비디오 클립에 대한 간단한 한두 문장 설명을 제공하세요.
> 
> 입력 비디오 클립은 비디오에서 샘플링된 프레임 그리드를 포함하는
> 일련의 이미지로 제시됩니다. 각 이미지의 프레임은 왼쪽에서 오른쪽,
> 위에서 아래로 시간 순서대로 구성되어 있습니다.
> 
> 1. 클립의 장르를 Science Fiction, Comedy, Mystery and 
>    Suspense, Horror, Drama, Documentary, 또는 Other로 분류하세요.
> 
> 2. 설명은 클립의 중요한 시각적 정보를 설명하는 이야기의 내레이션이어야 
>    합니다. 내레이션 스타일을 결정하기 위해 장르를 사용하세요. 비디오 
>    프레임이나 클립을 언급하지 말고, 대신 클립의 이야기를 관찰하세요. 
>    클립에서 보이는 것만 내레이션하세요. 한 가지 이상의 행동이 있더라도 
>    클립의 주요 인물들의 모든 행동을 포함하도록 하세요.
>    예시: "여자가 책을 선반에 놓고 뒤에 서 있는 소녀를 향해 
>    돌아섭니다..." 출력을 제공하기 전에 모든 프레임을 철저히 
>    검토하도록 주의하세요. 서문은 건너뛰고 바로 설명으로 
>    들어가세요.
> 
> 3. 내레이션할 내용이 없다면 클립에 보이는 것을 설명하세요.
> 
> 4. JSON 형식으로만 출력을 제공하세요
> 
> 단계별로 생각하세요
> ```

In [ ]:
IMAGES = video['shots'].shots[61]['composite_images']

PROMPT = f"""Provide a concise one or two sentence description of a video clip.
        The input video clip is presented as a sequence of images each 
        containing a grid of frames sampled from a video.  The frames in each
        image are organized in time sequence reading left to right and top to bottom.  
        1. Classify the genre of the clip as Science Fiction, Comedy, Mystery and 
           Suspense, Horror, Drama, Documentary, or Other.
        2. The description should be a narration of a story to describe the important 
           visual information in the clip.  Use the genre to determine 
           the style of the narration.  Do not refer to video frames or clips, instead 
           observe the story in the clip.  Only narrate what you see in the
           clip.  Ensure that you include all the actions of
           the main people in the clip even if there are more than one action.  
           Example: "the woman puts the book on the shelf and turns to face
           the girl standing behind her..." You are careful to throuroughly 
           examine all the frames before giving the output.  Skip the 
           preamble; go straight into the description.
        3. If there is nothing to narrate describe what is shown in the clip.
        3. Provide the output in JSON only
        Think step by step. Answer in Korean."""


response = generate_segment_description(IMAGES , PROMPT)
response_json = json.loads(response)

for image in IMAGES:
    display(DisplayImage(f"{image['file']}", width=600))

display(JSON(response_json, root="respone"))


🤔 이제 프레임 그리드로 몇 가지 간단한 프롬프트를 시도해보았으니, 결과의 품질에 대해 생각해 봅니다. 결과가 좋았던 부분도 있고, 부정확한 결과를 발견했을 수도 있습니다. 여기서 제시한 프롬프트는 매우 기본적입니다. 다양한 사용 사례에 맞게 결과를 조정하는 데 도움이 되는 많은 prompt engineering 기법들이 있습니다. 이러한 기법들에는 다음이 포함됩니다:

1. 클립의 대본, Rekognition Face Search API로 생성된 장면 속 캐릭터의 이름, 또는 이 시점까지의 비디오 플롯 요약과 같은 추가 컨텍스트 제공
2. 프롬프트를 여러 프롬프트로 나누는 prompt flows 사용. 예를 들어, 클립 분류를 위한 프롬프트와 장면 설명을 생성하기 위한 다른 프롬프트를 작성할 수 있습니다. 이 기법을 사용하면 이전 프롬프트의 결과를 기반으로 다운스트림 작업에 다른 프롬프트를 사용할 수 있고, 작업의 다른 부분에 다른 FM을 사용할 수 있다는 장점이 있습니다. Prompt flows는 비용을 제어하고 품질을 향상시키는 데 도움이 될 수 있습니다.
3. FM에 역할과 전반적인 지침을 제공하기 위한 시스템 프롬프트 추가
4. FM이 성능이 좋지 않은 상황을 수정하기 위한 부정적 프롬프트 사용
6. 이미지 해상도를 높이기 위해 그리드에 패킹되는 프레임 수 감소
7. 사용 사례에 맞게 세그먼테이션(또는 프레임 그룹) 변경. 예를 들어, 비디오의 프리롤, 메인 콘텐츠, 크레딧에 대해 수행하고 싶은 다른 작업이 있을 수 있습니다.

많은 가능성이 있습니다. 하지만 간단한 프롬프트로도 유용한 결과를 얻을 수 있습니다.

# 다음은 무엇인가요?

### 워크숍의 나머지 부분에서 사용할 수 있도록 비디오 메타데이터 저장

In [ ]:
%store video

### 워크숍의 다음 섹션으로 계속하기

워크숍의 다음 섹션에서는 Amazon Transcribe를 사용하여 비디오 내 음성에서 대본을 생성할 것입니다. 대본은 비디오 내러티브의 맥락을 기반으로 세그먼트를 찾는 데 사용할 수 있는 추가 정보를 제공합니다. 또한 대본은 비디오 클립에 대한 프롬프트에서 더 많은 입력 컨텍스트를 제공하는 데 사용될 수 있습니다.

다음 노트북 [Audio Segments](./01B-audio-segments.ipynb)으로 이동하세요.